In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.data import Dataset, AUTOTUNE
from tensorflow import keras
import re
from keras import losses

2024-10-03 21:42:48.624460: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-03 21:42:48.631924: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-03 21:42:48.640618: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-03 21:42:48.643483: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-03 21:42:48.651188: I tensorflow/core/platform/cpu_feature_guar

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


I0000 00:00:1727980969.528527   10548 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727980969.547035   10548 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727980969.547067   10548 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
raw = open('data2.txt', mode='r', encoding='utf-8').readlines()
data = []
for line in raw:
    if line != '\n' and 'Глава' not in line:
        data.append(' '.join(line.split()[1:]))
    
data = [line.replace('\n', ' ').replace('\xa0', ' ') for line in data]
text = ' '.join(data)

In [4]:
text[:100]

'завет ЗАВЕТ Моисея  В начале сотворил Бог небо и землю. Земля же была безвидна и пуста, и тьма над б'

In [5]:
def get_features_target(seq):
    features = seq[:-1]
    target = seq[1:]
    return features, target

BATCH_SIZE = 32

word_pattern = r'[^а-яА-ЯёЁ0-9 :,-]'
clear_text = re.sub(word_pattern, '', text)

alphabet = np.array(sorted(set(clear_text.split(' '))))

clear_text = np.array(clear_text.split(' '))
clear_text = clear_text[clear_text != '']

word_to_idx = {}
idx_to_word = {}

for idx, word in enumerate(alphabet):
    word_to_idx[word] = idx
    idx_to_word[idx] = word

text_idx = np.array([word_to_idx[word] for word in clear_text])
sequences = Dataset.from_tensor_slices(text_idx).batch(BATCH_SIZE, drop_remainder=True)
dataset = sequences.map(get_features_target)

data = dataset.batch(BATCH_SIZE, drop_remainder=True).repeat()
data = data.prefetch(AUTOTUNE)

I0000 00:00:1727980970.085963   10548 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727980970.086005   10548 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727980970.086015   10548 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727980970.168439   10548 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727980970.168484   10548 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-03

In [6]:
model = keras.Sequential([
    keras.layers.Embedding(len(alphabet), BATCH_SIZE),
    keras.layers.SimpleRNN(128, return_sequences=True, stateful=True),
    keras.layers.Dense(len(alphabet), activation='relu'),
])

model.compile(optimizer='adam', loss=losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(data, epochs=20, verbose=1, steps_per_epoch= len(sequences) // BATCH_SIZE)

Epoch 1/20


I0000 00:00:1727980977.224636   10684 service.cc:146] XLA service 0x7fd82c009b40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727980977.224673   10684 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4080 SUPER, Compute Capability 8.9
2024-10-03 21:42:57.244486: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-03 21:42:57.319596: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


 13/651 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.0251 - loss: 11.0997 

I0000 00:00:1727980978.012413   10684 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


651/651 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.0698 - loss: 9.3589
Epoch 2/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.0766 - loss: 8.8319
Epoch 3/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.0863 - loss: 8.6341
Epoch 4/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.0945 - loss: 8.5128
Epoch 5/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.1048 - loss: 8.4088
Epoch 6/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.1127 - loss: 8.3236
Epoch 7/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.1189 - loss: 8.2439
Epoch 8/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.1268 - loss: 8.1818
Epoch 9/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.1368 - loss: 7.9300
Epoch 10/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.1486 - loss: 7.7504
Epoch 11/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.1584 - loss: 7.6624
Epoch 12/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/st

In [7]:
def predict_next(sample, model, tokenizer, vocabulary, n_next, rnd_power, batch_size):
    sample_token = [tokenizer[char] for char in sample.split()]
    predicted = sample_token

    sample_tensor = tf.expand_dims(sample_token, 0)
    sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    
    for _ in range(n_next):
        cur = model(sample_tensor)
        cur = cur[0].numpy() / rnd_power
        cur = tf.random.categorical(cur, num_samples=1)[-1, 0].numpy()
        predicted.append(cur)
        sample_tensor = predicted[-99:]
        sample_tensor = tf.expand_dims([cur], 0)
        sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    res = [vocabulary[i] for i in predicted]
    generated = ' '.join(res)
    return generated

In [9]:
print(predict_next(
    sample='бог',
    model=model,
    tokenizer=word_to_idx,
    vocabulary=idx_to_word,
    n_next=20,
    rnd_power=0.6,
    batch_size=BATCH_SIZE,
))

бог мой Господь Бог твой за вас мы и что только вы ещё не только всё это И тогда он не


In [10]:
print(predict_next(
    sample='1',
    model=model,
    tokenizer=word_to_idx,
    vocabulary=idx_to_word,
    n_next=20,
    rnd_power=0.6,
    batch_size=BATCH_SIZE,
))

1 запретит вам И если мы уже не для своего но чтобы вы не в вас не из вас, ибо и
